In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [3]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 12254 (29.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10096 (24.3%) Users that have less than 1 test interactions


In [5]:
URM_all

<41629x24507 sparse matrix of type '<class 'numpy.int32'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [7]:
ICM_all = load_icm()
ICM_all

<27968x8 sparse matrix of type '<class 'numpy.intc'>'
	with 46181 stored elements in Compressed Sparse Row format>

In [8]:
profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)
profile_length, profile_length.shape

(array([ 2,  7, 20, ...,  7, 25,  4], dtype=int32), (41629,))

In [9]:
block_size = int(len(profile_length)*0.05)
block_size

2081

In [10]:
sorted_users = np.argsort(profile_length)
sorted_users

array([20814, 30938, 30960, ...,  9097, 30753, 38102], dtype=int64)

In [11]:
for group_id in range(0, 20):
    start_pos = group_id * block_size
    end_pos = min((group_id+1) * block_size, len(profile_length))

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]

    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id,
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))

Group 0, #users in group 2081, average p.len 1.27, median 1.0, min 0, max 2
Group 1, #users in group 2081, average p.len 2.29, median 2.0, min 2, max 3
Group 2, #users in group 2081, average p.len 3.00, median 3.0, min 3, max 3
Group 3, #users in group 2081, average p.len 3.60, median 4.0, min 3, max 4
Group 4, #users in group 2081, average p.len 4.00, median 4.0, min 4, max 4
Group 5, #users in group 2081, average p.len 4.61, median 5.0, min 4, max 5
Group 6, #users in group 2081, average p.len 5.00, median 5.0, min 5, max 5
Group 7, #users in group 2081, average p.len 5.54, median 6.0, min 5, max 6
Group 8, #users in group 2081, average p.len 6.00, median 6.0, min 6, max 6
Group 9, #users in group 2081, average p.len 6.67, median 7.0, min 6, max 7
Group 10, #users in group 2081, average p.len 7.05, median 7.0, min 7, max 8
Group 11, #users in group 2081, average p.len 8.00, median 8.0, min 8, max 8
Group 12, #users in group 2081, average p.len 8.68, median 9.0, min 8, max 9
Group 13,

In [12]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_class = SLIMElasticNetRecommender

import os

output_folder_path = "Experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [13]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...

C:\Users\Luca\miniconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\Recommenders\MatrixFactorization\Cython\MatrixFactorizationImpressions_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\Luca\miniconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\Recommenders\MatrixFactorization\Cython\MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\Luca\miniconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\


run_compile_all_cython: All files will be compiled using your current python environment: 'C:\Users\Luca\miniconda3\python.exe'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... PASS

Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... 
Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... PASS

Compiling [3/10]: Compute_Similarity_Cython.pyx... 
Compiling [3/10]: Compute_Similarity_Cython.pyx... PASS

Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... 
Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... PASS

Compiling [5/10]: Sparse_Matrix_Tree_CSR.pyx... 
Compiling [5/10]: Sparse_Matrix_Tree_CSR.pyx... PASS

Compiling [6/10]: Triangular_Matrix.pyx... 
Compiling [6/10]: Triangular_Matrix.pyx... PASS

Compiling [7/10]: CFW_DVV_Similarity_Cython_SGD.pyx... 
Compiling [7/10]: CFW_DVV_Similarity_Cython_SGD.pyx... PASS

Compiling [8/10]: CFW_D_Similarity_Cython_SGD.pyx... 
Compiling [8/10]: CFW_D_Si

In [14]:
#here we will save MAPs of different user groups
MAP_recommender_per_group = {}

#here we will save pairs label : recommender_object
recommender_object_dict = {}

## Recommenders to try:

In [15]:
#Done
from Recommenders.NonPersonalizedRecommender import TopPop

top_pop = TopPop(URM_train)
top_pop.fit()
recommender_object_dict["TOP_POP"] = top_pop

TopPopRecommender: URM Detected 252 ( 0.6%) users with no interactions.
TopPopRecommender: URM Detected 11728 (47.9%) items with no interactions.


In [16]:
#Done
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

itemknn_cf = ItemKNNCFRecommender(URM_train)
itemknn_cf.fit(topK = 100, shrink = 5, similarity = 'cosine', normalize = True)
recommender_object_dict["ItemKNNCF"] = itemknn_cf

ItemKNNCFRecommender: URM Detected 252 ( 0.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 11728 (47.9%) items with no interactions.
Similarity column 24507 (100.0%), 4325.89 column/sec. Elapsed time 5.67 sec


In [17]:
#Done
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3_beta = RP3betaRecommender(URM_train)
rp3_beta.fit(topK=167, alpha=1.0, beta=0.4520495673133021, implicit=True)

RP3betaRecommender: URM Detected 252 ( 0.6%) users with no interactions.
RP3betaRecommender: URM Detected 11728 (47.9%) items with no interactions.
RP3betaRecommender: Similarity column 24507 (100.0%), 1100.90 column/sec. Elapsed time 22.26 sec


In [18]:
#Done
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

pure_svd = PureSVDRecommender(URM_train)
pure_svd.fit(num_factors=31)
recommender_object_dict["PURE_SVD"] = pure_svd

PureSVDRecommender: URM Detected 252 ( 0.6%) users with no interactions.
PureSVDRecommender: URM Detected 11728 (47.9%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 1.68 sec


In [19]:
#Done
slim_en = SLIMElasticNetRecommender(URM_train)
slim_en.fit(topK=405, l1_ratio=0.0010299956370568744, alpha=0.01)
recommender_object_dict["SLIM_EN"] = slim_en

SLIMElasticNetRecommender: URM Detected 252 ( 0.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 11728 (47.9%) items with no interactions.
SLIMElasticNetRecommender: Processed 795 ( 3.2%) in 5.00 min. Items per second: 2.65
SLIMElasticNetRecommender: Processed 1539 ( 6.3%) in 10.01 min. Items per second: 2.56


KeyboardInterrupt: 

In [ ]:
#Done
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

slim_bpr = SLIM_BPR_Cython(URM_train)
slim_bpr.fit(epochs=462, sgd_mode = "sgd", topK = 50, lambda_i = 0.0002100158148046903, lambda_j = 0.00021427617376060016, learning_rate = 0.02543769736452639)
recommender_object_dict["SLIM_BPR"] = slim_bpr

## Plot results of models for different users groups

In [ ]:
# here we perform validation over different user groups for each model
# then we plot a graph to compare them
cutoff = 10

for group_id in range(0, 20):

    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]

    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id,
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))


    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]

    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

    for label, recommender in recommender_object_dict.items():
        result_df, _ = evaluator_validation.evaluateRecommender(recommender)
        if label in MAP_recommender_per_group:
            MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
        else:
            MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

_ = plt.figure(figsize=(16, 9))
for label, recommender in recommender_object_dict.items():
    results = MAP_recommender_per_group[label]
    plt.scatter(x=np.arange(0,len(results)), y=results, label=label)
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()